# Unit 4: Data Manipulation
---

1. [Working with dates](#section1)
2. [Simple Groupby](#section2)
3. [Sorting and filtering](#section3)
4. [Two or more attributes](#section4)
5. [Multiple aggregations](#section5)
6. [Tidy up](#section6)

<div>
<img src="https://github.com/nlihin/EDA-course/blob/main/images/CRISP-DM.png?raw=true" width="600"/>
</div>

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Still with airport wildlife strikes. 
[Some more info on the data](https://github.com/nlihin/data-analytics/blob/main/datasets/aircraft%20wildlife%20strikes%20fieldlist.pdf)

This time data from 2018-2023. This dataset was too big to upload to Github, so I split it into two parts:\
2018-2020\
2021-2023

In [ ]:
url1 = 'https://raw.githubusercontent.com/nlihin/data-analytics/main/datasets/aircraft%20wildlife%20strikes%202018-2020.csv'
url2 = 'https://raw.githubusercontent.com/nlihin/data-analytics/main/datasets/aircraft%20wildlife%20strikes%202021-2023.csv'
strike_df_18_20 = pd.read_csv(url1)
strike_df_21_23 = pd.read_csv(url2)

#### Combine the two files into one:
Since they have the same structure, `concat` can be used.
You will learn other options next year in DB course, but if you're curious, [look here](https://pandas.pydata.org/docs/user_guide/merging.html)

In [ ]:
strike_df = pd.concat([strike_df_18_20 ,strike_df_21_23]).reset_index()
strike_df.iloc[[0, 1, -2, -1]]

📝 Note: `reset_index` is important here, or else you get the former indices from both datasets (and then have multiple rows with the same index).  
Another way to achieve the same thing: use `ignore_index=True` inside `pd.concat()`.

In [ ]:
strike_df = pd.concat([strike_df_18_20, strike_df_21_23], ignore_index=True)

### <span style="color:blue"> Exercise:</span>
> 
> How do you suggest to check that the files have been combined? (Sanity check)

<a id='section1'></a>
## 1. Working with dates

We have a cell that holds the dates:

In [ ]:
strike_df[['INCIDENT_DATE']].tail()

But it's currently of type object:

In [ ]:
strike_df[['INCIDENT_DATE']].dtypes

We will change 'INCIDENT_DATE' into a `datetime` object 

In [ ]:
strike_df['date'] = pd.to_datetime(strike_df['INCIDENT_DATE'],format='%d/%m/%Y')

Did it work?

In [ ]:
strike_df[['date']].dtypes

In [ ]:
strike_df[['date','INCIDENT_DATE']]

Extract the month:

In [ ]:
strike_df['month'] = pd.DatetimeIndex(strike_df['date']).month

In [ ]:
strike_df.loc[1000:1010,['date','month','struck_parts','SPECIES']]

You can use any combination [from here](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)

Another example:

In [ ]:
strike_df['year-month'] = pd.DatetimeIndex(strike_df['date']).strftime('%Y-%m')
strike_df[["year-month",'date']].iloc[10:12]

### <span style="color:blue"> Exercise:</span>
> Extract the `year` and add it as a new column called `year` in `strike_df`
>
> Extract the name of the day and add it as a new column called `weekday` in `strike_df`
>
> Run the sanity check: `strike_df[["date","year","weekday"]]` 

In [ ]:
# sanity check
#strike_df[["date","year","weekday"]]

<a id='section2'></a>
## 2. Simple groupby

[from pandas user guide:](https://pandas.pydata.org/docs/user_guide/groupby.html)\
By “group by” we are referring to a process involving one or more of the following steps:

* Splitting the data into groups based on some criteria.

* Applying a function to each group independently.

* Combining the results into a data structure.



In [ ]:
grouped = strike_df.groupby('year')
grouped

The above split the data into groups\
Nothing happens here, since we didn't indicate what to do with each group\
But: no error. The split is valid :-)

Now lets perform a split and then an apply of an aggregation function

Count the number of non-missing (non-NaN) values in the 'WARNED' column each year:

In [ ]:
y_df = strike_df.groupby('year')[['WARNED']].count()

In [ ]:
#y_df[["year"]]

#### 📝 This format means `year` is now the index  
This means `strike_df[["year"]]` won't work anymore

##### If you plan to continue using this data and need the index as an attribute:

##### add `reset_index()` and then assign

In [ ]:
y_df = y_df.reset_index()
y_df

sum some data:

In [ ]:
strike_df["people_impact"] = strike_df[['NR_INJURIES', 'NR_FATALITIES']].sum(axis=1)

#### Understanding `axis=0` and `axis=1`

- `axis=0` → sum **down** (sum over rows — this is the default).
- `axis=1` → sum **across** (sum values **within the same row**, across different columns).

### <span style="color:blue"> Exercise:</span>
> What is the total number of people impacted (the `people_impact`) in each month?
>
> What is the mean (average) of people impacted (the `people_impact`) each year?


##### Now let's see these on a graph.
**📝 Note: You DO NOT need to groupby the data yourself**

In [ ]:
plt.figure(figsize=(6,3))
strike_df['year'] = pd.DatetimeIndex(strike_df['date']).year
sns.barplot(data=strike_df, y ='people_impact', x = 'year',errorbar=None)

In [ ]:
plt.figure(figsize=(6,3))
sns.barplot(data=strike_df, y ='people_impact', x = 'month',estimator = sum,  errorbar=None)

<a id='section3'></a>
## 3. Sorting and filtering or: when we MUST use `groupby`

We created a barplot showing how many people where impacted:  
(a) each year  
(b) each month  
now we want:  
(c) by each bird

<div>
<img src="https://github.com/nlihin/EDA-course/blob/main/images/people_impact_summary.png?raw=true" >
</div>

### <span style="color:blue"> (Thought) Exercise:</span>
> How can you do that?
>



Let's groupby bird `SPECIES`:

In [ ]:
strike_df.groupby(['SPECIES'])[['people_impact']].sum().sort_values(by="people_impact", ascending = False)\
                                                                                             .reset_index()

Most birds don't impact people. Filter out those who don't using `Filter`\
& then groupby again. You need to groupby twice:
* for the filter
* for the result after the filter  

Give this new slice of the dataframe a name: `bird_df`

In [ ]:
bird_df = strike_df.groupby('SPECIES').filter(lambda x: x['people_impact'].sum() > 0) \
                            .groupby('SPECIES')[['people_impact']].sum() \
                            .sort_values(by="people_impact", ascending = False ) \
                            .reset_index()                            
bird_df

##### Now we can create a bar plot
##### 📝 **Note: We had to perform a `groupby` and apply some filtering.**
This is because the original data does not contain a direct column showing the number of people injured in each accident involving each bird species.


In [ ]:
plt.figure(figsize=(5,3))
sns.barplot(data = bird_df.head(6), x = "SPECIES", y = "people_impact")
plt.xticks(rotation=45, ha='right')
plt.show()

<a id='section4'></a>
## 4. Groupby on two or more attributes

Now, groupby `month` and `year`

In [ ]:
strike_grpd = strike_df.groupby(['month','year'])[['people_impact', 'struck_parts']].sum()\
.reset_index()
strike_grpd

### <span style="color:blue"> Exercise:</span>
> 
> 1. what will happen if we switch the order of the indexes: `['month', 'year']`?
>
> 2. Create a barplot for x="year", y="people_impact", alongside the lineplot (see the help below)

Some help with 2:

In [ ]:
fig, axes = plt.subplots(figsize=(12, 4), ncols=2)
#WRITE YOUR CODE HERE
sns.lineplot(x="year", y="struck_parts",  data=strike_df,errorbar=None, estimator = sum, ax=axes[1])
plt.show()

<a id='section5'></a>
## 5. Multiple aggregations

In [ ]:
strike_df['TIME_OF_DAY'].unique()

In [ ]:
strike_group = strike_df.groupby('TIME_OF_DAY').\
agg({'people_impact': ['mean', 'median', 'max','count','sum'], 
     'TIME_OF_DAY':['count'],
     'SKY':['count'],
     'date':['first', 'last']})

strike_group = strike_group.reset_index()
strike_group

<a id='section6'></a>
## 6. Tidy your output

If you want to access the data and not deal with a multi-index, flatten the data by dropping a level and rename the columns:

In [ ]:
strike_group.columns

Each column currently has a multi-index, that is - several levels (two levels in our case).
We use [droplevel](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.droplevel.html) to remove one of the indexes.\
`droplevel(level, axis=0)`\
`level` - the position of the index to drop. The topmost or leftmost index is 0.\
`axis` - 0 removes a level in the columns, 1 removes a level in the rows.\
In our case, we have two rows of index, so `axis = 1`.

In [ ]:
strike_group = strike_group.droplevel(0, axis=1) 
strike_group

Rename the columns

In [ ]:
strike_group.columns = ['time of day','impact_mean','impact_median','impact_max','impact_count',
                        'impact_sum','time_of_day_count','sky_count','date_first','date_last']
strike_group

# Save as a pickle

In [ ]:
strike_df.to_pickle("pickled_strike")

Read the file:

In [ ]:
strike_df = pd.read_pickle("pickled_strike")

Pickling the file allows us to save it with all of the changes we made in python. It is also supposed to be faster to read&write

# Summary:


---

>* `concat()` - combine dataframes with overlapping columns [documentation](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)
>
>* `dtypes` - returns the data type of a series (a column in a dataframe)
>
>* `pd.to_datetime(df['date'])` - changes the attribute type to datetime
>
>* `pd.DatetimeIndex(df['date']).month` - extracts the month from the datatime attribute
>
>*  `pd.DatetimeIndex(strike_df['date']).strftime('%y-%m-%w')` - create a string representing the time. use %y, %a, %A etc..  
>
>* `groupby()` - group according to the columns specified
>
>* `reset_index()` reset index to a numerical index beginning at 0
>
>* `apply` - applies a function on each row (axis =0) in the dataframe. Change to (axis = 1) to apply the function on each column [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html#pandas.DataFrame.apply)
>
>* `lambda` - small anonymous function
>
>* `agg` - apply multiple functions at once, one for each specified column [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html)
>
>* `droplevel(0, axis = 1)` - drops the highest (first) level in the column index of a multi-index dataframe
>
---

#### This was a lot of information.

#### Keep your balance. Practice. You will make it.

<div>
<img src="https://raw.githubusercontent.com/nlihin/data-analytics/main/images/balance.jpg" width="500"/>
</div>

Photo by <a href="https://unsplash.com/@martinsanchez?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Martin Sanchez</a> on <a href="https://unsplash.com/s/photos/perfect-balance?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>
  